In [19]:
import numpy as np
import pandas as pd
import csv
from datetime import datetime

Признаки

Клиент

Число посещений

Число покупок со скидкой 

Макс/Мин/Среднее/медиана/дисперсия дней между посещениями

Макс/Мин/Среднее/медиана/дисперсия число товаров за посещение

Макс/Мин/Среднее/медиана/дисперсия число покупок за посещение

Макс/Мин/Среднее/медиана/дисперсия число уникальных товаров за посещение

Макс/Мин/Среднее/медиана/дисперсия суммы покупок за посещение

Макс/Мин/Среднее/медиана/дисперсия скидки покупок за посещение

Макс/Мин/Среднее/медиана/дисперсия regular_points_received

Макс/Мин/Среднее/медиана/дисперсия express_points_received

Макс/Мин/Среднее/медиана/дисперсия regular_points_spent

Макс/Мин/Среднее/медиана/дисперсия express_points_spent

Макс/Мин/Среднее/медиана/дисперсия цен одной купленной единицы



In [10]:
PATH='E:/Source data X5 hero/retailhero-uplift/data'
'''Блок парня с форума, пока коментим'''

# # Чтение данных
# df_clients = pd.read_csv(PATH + '/clients.csv', index_col='client_id')

# df_train = pd.read_csv(PATH + '/uplift_train.csv', index_col='client_id')
# df_test = pd.read_csv(PATH + '/uplift_test.csv', index_col='client_id')

# # Извлечение признаков

# # df_features = df_clients.copy()
# # df_features['first_issue_time'] = pd.to_datetime(df_features['first_issue_date']).astype(int) / 10 ** 9
# # df_features['first_redeem_time'] = pd.to_datetime(df_features['first_redeem_date']).astype(int) / 10 ** 9
# # df_features['issue_redeem_delay'] = df_features['first_redeem_time'] - df_features['first_issue_time']
# # df_features = df_features.drop(['first_issue_date', 'first_redeem_date'], axis=1)

# # indices_train = df_train.index
# # indices_test = df_test.index
# # indices_learn, indices_valid = train_test_split(df_train.index, test_size=0.3, random_state=123)

'Блок парня с форума, пока коментим'

In [11]:


def getstuff(filename):
    with open(filename, "r") as csvfile:
        datareader = csv.reader(csvfile)
        yield next(datareader)  # yield the header row
        count = 0
        for row in datareader:
            yield row
            count += 1
            #return

In [12]:
i=0
filename = PATH + '/purchases.csv'
for row in getstuff(filename):
    print(row)
    i+=1
    if i>2:
        break

['client_id', 'transaction_id', 'transaction_datetime', 'regular_points_received', 'express_points_received', 'regular_points_spent', 'express_points_spent', 'purchase_sum', 'store_id', 'product_id', 'product_quantity', 'trn_sum_from_iss', 'trn_sum_from_red']
['000012768d', '7e3e2e3984', '2018-12-01 07:12:45', '10.0', '0.0', '0.0', '0.0', '1007.0', '54a4a11a29', '9a80204f78', '2.0', '80.0', '']
['000012768d', '7e3e2e3984', '2018-12-01 07:12:45', '10.0', '0.0', '0.0', '0.0', '1007.0', '54a4a11a29', 'da89ebd374', '1.0', '65.0', '']


In [13]:
list_func = [np.max, np.min, np.mean, np.median, np.std]

def create_feature(list_func,lists_of_feature):
    result_list=[]
    for feature in lists_of_feature:
        for func in list_func:
            result_list.append(func(feature))
    return result_list

def clear_lists(lists_of_feature):
    for feature_list in lists_of_feature:
        feature_list=[]

In [14]:
%%time
id_of_row = 0
purchase=[]
for row in getstuff(filename):
    # пропускаем header
    if id_of_row >= 1:
        # читаем текущие id клиента, транзакции, даты
        curr_client_id = row[0]
        curr_transaction_id = row[1]
        curr_transaction_datetime = row[2]
        
        # первая строка рассматривается отдельнл
        if id_of_row==1:
            # инициализируем предыдущие значения  id клиента, транзакции
            prev_client_id = curr_client_id
            prev_transaction_id = curr_transaction_id
            prev_transaction_datetime = curr_transaction_datetime
            # инициализируем количество визитов в магазин единицей
            n_visits=1
            
            # инициализируем количество визитов в магазин со скидкой
            n_red_visits=0
            
            # инициализируем стоимость отдельной позиции и количество товаров в чеке нулями
            sum_trn_sum_from_iss = 0
            sum_trn_sum_from_red = 0
            sum_product_quantity = 0
            sum_product_unique = 0
            
            # инициализируем листы фичей
            list_client_day = []                #дни посещений
            list_delay_days = []                #промужуток дней между посещений
            
            list_regular_points_received = []   #список полученных "регулярных"  баллов
            list_express_points_received = []   #список полученных "экспресс" баллов
            list_regular_points_spent = []      #список потраченных "регулярных"  баллов
            list_express_points_spent = []      #список потраченных "регулярных"  баллов
            
            list_purchase_sum = []              #список потраченных денег на покупки
            list_sum_product_quantity = []      #список числа купленных товаров
            list_sum_product_unique = []        #список числа уникальных купленных товаров
            
            list_trn_sum_from_iss = []          #список цен купленных позиций
            list_trn_sum_from_red = []          #список цен оплаченных бонусами
            list_price_from_iss = []            #список цен за одну единицу товара
            

        # Проверяем уникальность визита в магазин. 
        # Если текущая транзакция равна предыдущей,
        # вычисляем признаки текущего посещения
        if curr_transaction_id == prev_transaction_id:
            
            #считываем признаки для текущей позиции в чеке, которые просто повторяются
            regular_points_received = float(row[3])
            express_points_received = float(row[4])
            regular_points_spent = float(row[5])
            express_points_spent = float(row[6])
            purchase_sum = float(row[7])
            
            #число уникальных товаров +1
            sum_product_unique += 1
            
            #общее число товаров 
            if float(row[10])==0:
                quantity = 1.0
            else:
                quantity = float(row[10])    
            sum_product_quantity += quantity
            
            #стоимость позиции
            trn_sum_from_iss = float(row[11])
            list_trn_sum_from_iss.append(trn_sum_from_iss)
            
            #стоимость товара
            price_from_iss = trn_sum_from_iss/quantity
            list_price_from_iss.append(price_from_iss)
            
            #стоимость позизии оплаченной бонусами
            if row[12]:
                trn_sum_from_red = float(row[12])
            else:
                trn_sum_from_red = 0 
            list_trn_sum_from_red.append(trn_sum_from_red)
    
        else:
            # Это новый визит!
            # Подведем итоги прошлого визита.
            
            #Добавляем день посещения
            list_client_day.append(prev_transaction_datetime)
            
            #Добавляем информацию о бонусах за посещение
            list_regular_points_received.append(regular_points_received)
            list_express_points_received.append(express_points_received)
            list_regular_points_spent.append(regular_points_spent)
            list_express_points_spent.append(express_points_spent)
            
            #Добавляем информацию о количестве и качестве покупок
            list_purchase_sum.append(purchase_sum)
            list_sum_product_quantity.append(sum_product_quantity)
            list_sum_product_unique.append(sum_product_unique)
            
            
            # Проверяем id клиента
            if curr_client_id == prev_client_id:
                
                # Это тот же клиент! Число посещений +1
                n_visits+=1
                
                # Если нужно добавляем в число "красных" визитов  
                if 0 not in list_trn_sum_from_red:
                    n_red_visits+=1
            else:
                # Это другой клиент! Подводим итоги
                
                # Добавляем информацию о частоте посещения магазина
                if len(list_client_day)==1:
                    
                    # Если у клиента только одно посещение помечаем его -1
                    list_delay_days.append(-1)
                else:
                    # Вычисляем дилэй между ближайшими посещениями
                    for i in range(len(list_client_day)-1):
                        date_0 = datetime.strptime(list_client_day[i], '%Y-%m-%d %H:%M:%S')
                        date_1 = datetime.strptime(list_client_day[i+1], '%Y-%m-%d %H:%M:%S')
                        delay = (date_1 - date_0).days
                        
                        
                        list_delay_days.append(delay)
                    
                # Собираем список из списков фичей - 11 штук !
                lists_of_feature = [list_delay_days,
                                    
                                    list_regular_points_received,
                                    list_express_points_received,
                                    list_regular_points_spent,
                                    list_express_points_spent,
                                    
                                    list_purchase_sum,
                                    list_sum_product_quantity,
                                    list_sum_product_unique,
                                    
                                    list_trn_sum_from_iss,
                                    list_trn_sum_from_red,
                                    list_price_from_iss
                                   ]
                
                
                # Собираем итоговые признаки клиента
                # Инициалзируем лист который описывает клиента
                push=[]
                
                # Добавляем id, количество посещений, количество "красных" посещений
                push.extend([prev_client_id, n_visits, n_red_visits])
                
                # Создаем фичи для каждого листа с помощью 
                # ряда функций  
                # list_func = [np.max, np.min, np.mean, np.median, np.std]
                for feature in create_feature(list_func,lists_of_feature):
                    push.append(feature)
                
                purchase.append(push)
                
                # Подготавливаем данные для следующей итерации
                # Не забываем что сейчас уже читаем новую строку нового клиента!
                
                # Опустошаем использованные списки
                for item in lists_of_feature:
                    item=[]
                
                # Инициализируем данные для новой итерации
                n_visits = 1
                n_red_visits = 0
                
                # Число уникальных продуктов
                sum_product_unique = 1
                
                # Число товаров
                if float(row[10])==0:
                    quantity = 1.0
                else:
                    quantity = float(row[10])                    
                sum_product_quantity = quantity
                
                # Цена одной единицы товара
                price_from_iss = trn_sum_from_iss/quantity
                list_price_from_iss.append(price_from_iss)
            
                # Цена одной позиции
                trn_sum_from_iss = float(row[11])
                list_trn_sum_from_iss.append(trn_sum_from_iss)
                
                # Цена одной позиции без скидки
                if row[12]:
                    sum_trn_sum_from_red = float(row[12])
                else:
                    sum_trn_sum_from_red = 0
                list_trn_sum_from_red.append(trn_sum_from_red)
                
                # Обновляем предыдущие значения id клиента и транзакции
                prev_client_id = curr_client_id
                prev_transaction_id = curr_transaction_id
                prev_transaction_datetime = curr_transaction_datetime
    
    # Счетчик строк +1
    id_of_row+=1
    
    # Ограничимся строками 
    if id_of_row > 10000:
        break

Wall time: 10.2 s


In [24]:
name_of_abs_featur = ['client_id', 'n_visits', 'n_red_visits']
name_of_func = ['max', 'min', 'mean', 'median', 'std']
name_of_feature = ['delay_days',       
                    'regular_points_received',
                    'express_points_received',
                    'regular_points_spent',
                    'express_points_spent',

                    'purchase_sum',
                    'sum_product_quantity',
                    'sum_product_unique',

                    'trn_sum_from_iss',
                    'trn_sum_from_red',
                    'price_from_iss'
                   ]

columns=[]
for name in name_of_abs_featur:
    columns.append(name)
for feature in name_of_feature:
        for func in name_of_func:
            columns.append(func + '_' + feature)

print(f'Число признаков равно {len(columns)}')

Число признаков равно 58


In [25]:
data=pd.DataFrame(data=purchase, columns = columns)
data

,client_id,n_visits,n_red_visits,max_delay_days,min_delay_days,mean_delay_days,median_delay_days,std_delay_days,max_regular_points_received,min_regular_points_received,...,max_trn_sum_from_red,min_trn_sum_from_red,mean_trn_sum_from_red,median_trn_sum_from_red,std_trn_sum_from_red,max_price_from_iss,min_price_from_iss,mean_price_from_iss,median_price_from_iss,std_price_from_iss
0,000012768d,34,0,0,0,0.000000,0.0,0.000000,10.0,10.0,...,0.0,0.0,0.000000,0.0,0.000000,90.0,22.0,49.578947,49.0,21.238782
1,000036f903,158,0,0,-3,-0.013393,0.0,0.199998,10.0,1.2,...,0.0,0.0,0.000000,0.0,0.000000,90.0,22.0,51.375000,50.0,21.860948
2,000048b7a6,48,0,16,-3,0.021598,0.0,0.768977,10.0,1.2,...,0.0,0.0,0.000000,0.0,0.000000,160.0,17.0,50.363636,40.0,30.039185
3,000073194a,81,0,16,-20,0.003831,0.0,1.095089,10.0,0.3,...,0.0,0.0,0.000000,0.0,0.000000,160.0,8.5,48.200000,40.0,30.478611
4,00007c7133,76,0,16,-20,-0.004241,0.0,1.166750,10.0,0.3,...,0.0,0.0,0.000000,0.0,0.000000,160.0,5.0,45.918605,40.0,28.719358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,000aef1e0b,121,0,106,-107,-0.002749,0.0,2.739225,102.7,0.2,...,480.0,0.0,5.409336,0.0,28.868882,1070.0,0.0,65.861011,45.0,76.703549
80,000af410d8,24,0,106,-107,-0.002820,0.0,2.760281,102.7,0.2,...,480.0,0.0,5.323322,0.0,28.646432,1070.0,0.0,65.594670,45.0,76.175601
81,000b0559be,81,0,106,-107,-0.002882,0.0,2.779901,102.7,0.2,...,480.0,0.0,5.313933,0.0,28.622031,1070.0,0.0,65.549530,45.0,76.115974
82,000b397480,111,0,106,-107,-0.002937,0.0,2.798080,102.7,0.2,...,480.0,0.0,5.258290,0.0,28.476921,1070.0,0.0,65.831734,45.0,76.227644
